# How to fine-tune chat models

This notebook provides a step-by-step guide for our new `gpt-3.5-turbo` fine-tuning. We'll perform entity extraction using the [RecipeNLG dataset](https://github.com/Glorf/recipenlg), which provides various recipes and a list of extracted generic ingredients for each. This is a common dataset for named entity recognition (NER) tasks.

We will go through the following steps:

1. **Setup:** Loading our dataset and filtering down to one domain to fine-tune on.
2. **Data preparation:** Preparing your data for fine-tuning by creating training and validation examples, and uploading them to the `Files` endpoint.
3. **Create the fine-tune:** Creating your fine-tuned model.
4. **Use model for inference:** Using your fine-tuned model for inference on new inputs.

By the end of this you should be able to train, evaluate and deploy a fine-tuned `gpt-3.5-turbo` model.


## Setup

First we will import any required libraries and prepare our data.

Fine tuning works best when focused on a particular domain. It's important to make sure your dataset is both focused enough for the model to learn, but general enough that unseen examples won't be missed. Having this in mind, we have already extracted a subset from the RecipesNLG dataset to only contain documents from www.cookbooks.com.


In [ ]:
# make sure to use the latest version of the openai python package
!pip install --upgrade openai 

In [76]:
import json
import openai
import os
import pandas as pd
import requests
from pprint import pprint

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "")

In [2]:
# Read in the dataset we'll use for this task.
# This will be the RecipesNLG dataset, which we've cleaned to only contain documents from www.cookbooks.com
recipe_df = pd.read_csv("data/cookbook_recipes_nlg_10k.csv")

recipe_df.head()

,title,ingredients,directions,link,source,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,www.cookbooks.com,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,www.cookbooks.com,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,www.cookbooks.com,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,www.cookbooks.com,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,www.cookbooks.com,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."


## Data preparation

We'll begin by preparing our data. When fine-tuning with the `ChatCompletion` format, each training example is a simple list of `messages`. For example, an entry could look like:

```
[{'role': 'system',
  'content': 'You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided.'},

 {'role': 'user',
  'content': 'Title: No-Bake Nut Cookies\n\nIngredients: ["1 c. firmly packed brown sugar", "1/2 c. evaporated milk", "1/2 tsp. vanilla", "1/2 c. broken nuts (pecans)", "2 Tbsp. butter or margarine", "3 1/2 c. bite size shredded rice biscuits"]\n\nGeneric ingredients: '},

 {'role': 'assistant',
  'content': '["brown sugar", "milk", "vanilla", "nuts", "butter", "bite size shredded rice biscuits"]'}]
```

During the training process this conversation will be split, with the final entry being the `completion` that the model will produce, and the remainder of the `messages` acting as the prompt. Consider this when building your training examples - if your model will act on multi-turn conversations, then please provide representative examples so it doesn't perform poorly when the conversation starts to expand.

For fine-tuning with `ChatCompletion` you can begin with even 30-50 well-pruned examples. You should see performance continue to scale linearly as you increase the size of the training set, but your jobs will also take longer.

Please note that currently there is a 4096 token limit for each training example. Anything longer than this will be truncated at 4096 tokens.


In [17]:
training_data = []

system_message = "You are a helpful recipe assistant. You are to extract the generic ingredients from each of the recipes provided."


def create_user_message(row):
    return f"""Title: {row['title']}\n\nIngredients: {row['ingredients']}\n\nGeneric ingredients: """


# Take first 100 records for training
for x, y in recipe_df.head(100).iterrows():
    training_message = []
    training_message.append({"role": "system", "content": system_message})

    user_message = create_user_message(y)
    training_message.append({"role": "user", "content": user_message})

    training_message.append({"role": "assistant", "content": y["NER"]})
    training_message_dict = {"messages": training_message}
    training_data.append(training_message_dict)

In [18]:
validation_data = []

# We'll pick a test set from further on in the dataset
test_df = recipe_df.loc[100:200]

for x, y in test_df.iterrows():
    validation_message = []
    validation_message.append({"role": "system", "content": system_message})

    user_message = create_user_message(y)
    validation_message.append({"role": "user", "content": user_message})

    validation_message.append({"role": "assistant", "content": y["NER"]})
    validation_message_dict = {"messages": validation_message}
    validation_data.append(validation_message_dict)

We then need to export these as `.jsonl` files, with each row being one training example.


In [19]:
def dicts_to_jsonl(data_list: list, filename: str) -> None:
    """
    Method saves list of dicts into jsonl file.
    :param data: (list) list of dicts to be stored,
    :param filename: (str) path to the output file. If suffix .jsonl is not given then methods appends
        .jsonl suffix into the file.
    """
    sjsonl = ".jsonl"

    # Check filename
    if not filename.endswith(sjsonl):
        filename = filename + ".jsonl"

    # Save data
    with open(filename, "w") as out:
        for ddict in data_list:
            jout = json.dumps(ddict) + "\n"
            out.write(jout)

In [20]:
# Save training_data to JSONL
training_file_name = "tmp_recipe_finetune_training"
dicts_to_jsonl(training_data, training_file_name)

# Save validation_data to JSONL
validation_file_name = "tmp_recipe_finetune_validation"
dicts_to_jsonl(validation_data, validation_file_name)

### Upload files

You can then upload the files to our `Files` endpoint to be used by the fine-tuned model.


In [21]:
training_response = openai.File.create(
    file=open(training_file_name + ".jsonl", "rb"), purpose="fine-tune"
)
training_file_id = training_response["id"]

validation_response = openai.File.create(
    file=open(validation_file_name + ".jsonl", "rb"), purpose="fine-tune"
)
validation_file_id = validation_response["id"]

print("Training file id:", training_file_id)
print("Validation file id:", validation_file_id)

Training file id: file-XMhftmLXyyTvvmiLpMRMIAcL
Validation file id: file-kZz433aerpPIMADZF7xC5NKd


### Create fine-tune job

Now we can create our fine-tuning job with the generated files and an optional suffix to identify the model. The response will contain an `id` which you can use to retrieve updates on the job.

Note: The files have to first be processed by our system, so you might get a `File is not ready` error. In that case, simply retry a few minutes later.


In [24]:
suffix_name = "recipe-ner"


response = openai.FineTuningJob.create(
    training_file=training_file_id,
    validation_file=validation_file_id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)

job_id = response["id"]

print(response)

{
  "object": "fine_tuning.job",
  "id": "ftjob-ksOzx7zjpsrADZfhB5eyfB0Z",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692734343,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-l89177bnhkme4a44292n5r3j",
  "result_files": [],
  "status": "created",
  "validation_file": "file-kZz433aerpPIMADZF7xC5NKd",
  "training_file": "file-XMhftmLXyyTvvmiLpMRMIAcL",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}


#### Check job status

You can make a `GET` request to the `https://api.openai.com/v1/alpha/fine-tunes` endpoint to list your alpha fine-tune jobs. In this instance you'll want to check that the ID you got from the previous step ends up as `status: succeeded`.

Once it is completed, you can use the `result_files` to sample the results from the validation set (if you uploaded one), and use the ID from the `fine_tuned_model` parameter to invoke your trained model.


In [59]:
response = openai.FineTuningJob.retrieve(job_id)
print(response)

{
  "object": "fine_tuning.job",
  "id": "ftjob-ksOzx7zjpsrADZfhB5eyfB0Z",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692734343,
  "finished_at": 1692735182,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:openai:recipe-ner:7qS2GFaX",
  "organization_id": "org-l89177bnhkme4a44292n5r3j",
  "result_files": [
    "file-Tjt0E6BvQ846m75gYxtWiRzb"
  ],
  "status": "succeeded",
  "validation_file": "file-kZz433aerpPIMADZF7xC5NKd",
  "training_file": "file-XMhftmLXyyTvvmiLpMRMIAcL",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 39687
}


We can track the progress of the fine-tune with the events endpoint. You can rerun the cell below a few times until the fine-tune is ready. 


In [69]:
response = openai.FineTuningJob.list_events(id=job_id, limit=50)

events = response["data"]
events.reverse()

for event in events:
    print(event["message"])

Created fine-tune: ftjob-ksOzx7zjpsrADZfhB5eyfB0Z
Fine tuning job started
Step 10: training loss=0.16
Step 20: training loss=0.14
Step 30: training loss=0.30
Step 40: training loss=0.17
Step 50: training loss=0.03
Step 60: training loss=0.55
Step 70: training loss=0.09
Step 80: training loss=0.00
Step 90: training loss=0.15
Step 100: training loss=0.06
Step 110: training loss=0.03
Step 120: training loss=0.04
Step 130: training loss=0.21
Step 140: training loss=0.00
Step 150: training loss=0.02
Step 160: training loss=0.00
Step 170: training loss=0.00
Step 180: training loss=0.19
Step 190: training loss=0.55
Step 200: training loss=0.01
Step 210: training loss=0.00
Step 220: training loss=0.00
Step 230: training loss=0.00
Step 240: training loss=0.00
Step 250: training loss=0.00
Step 260: training loss=0.36
Step 270: training loss=0.16
Step 280: training loss=0.01
Step 290: training loss=0.04
Step 300: training loss=0.53
New fine-tuned model created: ft:gpt-3.5-turbo-0613:openai:recipe

Now that it's done, we can get a fine-tuned model ID from the job


In [81]:
response = openai.FineTuningJob.retrieve(job_id)
fine_tuned_model_id = response["fine_tuned_model"]

print(response)
print("\nFine-tuned model id:", fine_tuned_model_id)

{
  "object": "fine_tuning.job",
  "id": "ftjob-ksOzx7zjpsrADZfhB5eyfB0Z",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692734343,
  "finished_at": 1692735182,
  "fine_tuned_model": "ft:gpt-3.5-turbo-0613:openai:recipe-ner:7qS2GFaX",
  "organization_id": "org-l89177bnhkme4a44292n5r3j",
  "result_files": [
    "file-Tjt0E6BvQ846m75gYxtWiRzb"
  ],
  "status": "succeeded",
  "validation_file": "file-kZz433aerpPIMADZF7xC5NKd",
  "training_file": "file-XMhftmLXyyTvvmiLpMRMIAcL",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": 39687
}

Fine-tuned model id: ft:gpt-3.5-turbo-0613:openai:recipe-ner:7qS2GFaX


## Generate with the fine-tuned model

The last step is to use your fine-tuned model for inference. Similar to the classic `FineTuning`, you simply call `ChatCompletions` with your new fine-tuned model name filling the `model` parameter.

In [87]:
test_row = test_df.iloc[0]
test_messages = []
test_messages.append({"role": "system", "content": system_message})
user_message = create_user_message(test_row)
test_messages.append({"role": "user", "content": create_user_message(test_row)})

pprint(test_messages)

[{'content': 'You are a helpful recipe assistant. You are to extract the '
             'generic ingredients from each of the recipes provided.',
  'role': 'system'},
 {'content': 'Title: Pancakes\n'
             '\n'
             'Ingredients: ["1 c. flour", "1 tsp. soda", "1 tsp. salt", "1 '
             'Tbsp. sugar", "1 egg", "3 Tbsp. margarine, melted", "1 c. '
             'buttermilk"]\n'
             '\n'
             'Generic ingredients: ',
  'role': 'user'}]


In [88]:
response = openai.ChatCompletion.create(
    model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=500
)
print(response["choices"][0]["message"]["content"])

["flour", "soda", "salt", "sugar", "egg", "margarine", "buttermilk"]


## Conclusion

Congratulations, you are now ready to fine-tune your own models using the `ChatCompletion` format! We look forward to seeing what you build
